In [11]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd


url = "https://www.fmkorea.com/search.php?mid=stock&category=2997203870&listStyle=list&search_keyword=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&search_target=title_content&page=8"

all_data = []

try:
    res = requests.get(url)
    soup = bs(res.text, "lxml")
    rows = soup.find_all("tr")

    for r in rows:
        cols = r.find_all("td")
        if not cols:
            continue
        row_data = [c.get_text(strip=True) for c in cols]
        all_data.append(row_data)

except Exception as e:
    print(f"오류 발생: {e}")

# 3. 데이터프레임으로 변환 (컬럼 개수가 다를 수 있으므로 기본 출력)
if all_data:
    # 최대 컬럼 수에 맞춰 데이터프레임 생성
    df = pd.DataFrame(all_data)
    print(f"--- 전체 수집 결과 (총 {len(df)}행) ---")
    display(df)
else:
    print("HTML 응답은 성공했으나 표(tr) 데이터를 찾지 못했습니다.")
    print("응답 본문 앞부분 일부:", res.text[:500])

--- 전체 수집 결과 (총 23행) ---


,0,1,2,3,4,5
0,공지 더 보기(-3개),None,None,None,None,None
1,국내주식,"삼성, 엑시노스 2800로 커스텀 CPU 복귀 가능성... 퀄컴 독재 탈출할까2",sadsdl,2026.01.13,229,5
2,국내주식,"TSMC 2나노 공정 가격 급등에 퀄컴, '삼성 파운드리'로 눈 돌려…AI 칩 쟁탈전1",sadsdl,2026.01.13,466,12
3,국내주식,엑시노스 2700 사양 유출…2나노 SF2P·ARM C2 코어로 반격 나선 삼성3,sadsdl,2026.01.13,714,4
4,AD,신청하기,None,None,None,None
5,AD,신청하기,None,None,None,None
6,국내주식,삼성전자·기후부 손잡고 최전방 GOP에 안정적 물 공급2,sadsdl,2026.01.13,202,5
7,국내주식,원익홀딩스 차트분석요청9,지지와저항,2026.01.13,929,12
8,국내주식,네이버 최근 한달간 수급8,중징,2026.01.13,1097,11
9,국내주식,AI가 신탁 내림1,30만전자,2026.01.13,379,1


In [ ]:
print("test")

In [9]:
import time
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

BASE_URL = "https://www.fmkorea.com/search.php"
BASE_PARAMS = {
    "mid": "stock",
    "category": "2997203870",
    "search_keyword": "삼성전자",
    "search_target": "title_content",
    "listStyle": "list",
}

SLEEP_SEC = 2
MAX_RETRY = 3

def parse_one_page(html: str):
    soup = bs(html, "html.parser")
    table = soup.select_one("table.bd_lst.bd_tb_lst.bd_tb")
    tbody = table.select_one("tbody") if table else None
    rows = tbody.select("tr") if tbody else []

    out = []
    for tr in rows:
        td_cate = tr.select_one("td.cate a")
        td_title_a = tr.select_one("td.title a.hx")
        td_author = tr.select_one("td.author a")
        td_time = tr.select_one("td.time")
        tds_mno = tr.select("td.m_no")

        if not (td_cate and td_title_a and td_author and td_time and len(tds_mno) >= 2):
            continue

        views = tds_mno[0].get_text(strip=True)
        votes = tds_mno[1].get_text(strip=True)

        out.append({
            "탭": td_cate.get_text(strip=True),
            "제목": td_title_a.get_text(" ", strip=True),
            "글쓴이": td_author.get_text(strip=True),
            "날짜": td_time.get_text(strip=True),
            "조회": int(views.replace(",", "")) if views else None,
            "추천": int(votes.replace(",", "")) if votes else None,
        })
    return out

all_rows = []

with requests.Session() as s:
    for page in range(1, 11):  # 테스트용
        params = dict(BASE_PARAMS)
        params["page"] = page

        ok = False
        for attempt in range(1, MAX_RETRY + 1):
            try:
                r = s.get(BASE_URL, params=params, timeout=200)
                r.raise_for_status()  # 4xx/5xx면 예외 발생 [web:142]

                all_rows.extend(parse_one_page(r.text))
                ok = True
                break

            except requests.RequestException as e:
                # 실패하면 더 길게 쉬었다가 재시도(점점 증가) = backoff [web:133]
                wait = 60 * attempt   # 60초, 120초, 180초...
                print(f"[FAIL] page={page} attempt={attempt}/{MAX_RETRY} err={e}")
                print(f"-> {wait}초 쉬고 재시도")
                time.sleep(wait)

        if ok:
            print(f"[OK] page={page} total_rows={len(all_rows)}")
        else:
            print(f"[SKIP] page={page} (재시도 {MAX_RETRY}회 실패)")

        time.sleep(SLEEP_SEC)

df = pd.DataFrame(all_rows)
df.to_csv("fmkorea_search_page1_500.csv", index=False, encoding="utf-8-sig")
print("saved:", len(df))


[OK] page=1 total_rows=20
[OK] page=2 total_rows=40
[OK] page=3 total_rows=60
[OK] page=4 total_rows=80
[OK] page=5 total_rows=100
[OK] page=6 total_rows=120
[OK] page=7 total_rows=140
[OK] page=8 total_rows=160
[OK] page=9 total_rows=180
[OK] page=10 total_rows=200
saved: 200
